In [307]:
from libpgm.nodedata import NodeData
from libpgm.graphskeleton import GraphSkeleton
from libpgm.discretebayesiannetwork import DiscreteBayesianNetwork
from libpgm.lgbayesiannetwork import LGBayesianNetwork
from libpgm.hybayesiannetwork import HyBayesianNetwork
from libpgm.dyndiscbayesiannetwork import DynDiscBayesianNetwork
from libpgm.tablecpdfactorization import TableCPDFactorization
from libpgm.sampleaggregator import SampleAggregator
from libpgm.pgmlearner import PGMLearner

#create table
from prettytable import PrettyTable

#read csv
import pandas as pd

In [366]:
data=pd.read_csv("reviews_target.csv")

data.query('overall > 0',inplace=True)
print(len(data))
data.head()

190509


,id,hotel,reader,helpful,overall,value,room,location,clean,check,service,business,target
0,0,hotel_231505,-1,-1,5,5,5,5,5,5,5,5,good
1,1,hotel_231505,-1,-1,1,1,1,3,2,1,3,3,bad
2,2,hotel_231505,-1,-1,3,3,2,3,4,2,2,3,bad
3,3,hotel_231505,1,0,1,1,1,3,1,1,1,3,bad
4,4,hotel_231505,1,1,5,5,5,3,5,5,5,5,good


In [368]:
bad=data.query('target == " bad" and overall >0')
good=data.query('target == " good"')
bad_n=len(bad)
good_n=len(good)

bad_n



44577

In [384]:
#numero di volte per ogni valore di clean dato good
clean_g_1=data.query('target == " good" and clean == 1')
clean_g_2=data.query('target == " good" and clean == 2')
clean_g_3=data.query('target == " good" and clean == 3')
clean_g_4=data.query('target == " good" and clean == 4')
clean_g_5=data.query('target == " good" and clean == 5')
clean_g=[len(clean_g_1),len(clean_g_2),len(clean_g_3),len(clean_g_4),len(clean_g_5)]
clean_g

[38, 467, 4645, 42326, 98456]

In [385]:
#numero di volte per ogni valore di clean dato bad
clean_b_1=data.query('target == " bad" and clean == 1')
clean_b_2=data.query('target == " bad" and clean == 2')
clean_b_3=data.query('target == " bad" and clean == 3')
clean_b_4=data.query('target == " bad" and clean == 4')
clean_b_5=data.query('target == " bad" and clean == 5')
clean_b=[len(clean_b_1),len(clean_b_2),len(clean_b_3),len(clean_b_4),len(clean_b_5)]
clean_b

[7673, 9820, 14536, 9042, 3506]

In [386]:
#numero di volte per ogni valore di value dato good
value_g_1=data.query('target == " good" and value == 1')
value_g_2=data.query('target == " good" and value == 2')
value_g_3=data.query('target == " good" and value == 3')
value_g_4=data.query('target == " good" and value == 4')
value_g_5=data.query('target == " good" and value == 5')
value_g=[len(value_g_1),len(value_g_2),len(value_g_3),len(value_g_4),len(value_g_5)]

In [387]:
#numero di volte per ogni valore di value dato bad
value_b_1=data.query('target == " bad" and value == 1')
value_b_2=data.query('target == " bad" and value == 2')
value_b_3=data.query('target == " bad" and value == 3')
value_b_4=data.query('target == " bad" and value == 4')
value_b_5=data.query('target == " bad" and value == 5')
value_b=[len(value_b_1),len(value_b_2),len(value_b_3),len(value_b_4),len(value_b_5)]

In [389]:
#numero di volte per ogni valore di room dato good
room_g_1=data.query('target == " good" and room == 1')
room_g_2=data.query('target == " good" and room == 2')
room_g_3=data.query('target == " good" and room == 3')
room_g_4=data.query('target == " good" and room == 4')
room_g_5=data.query('target == " good" and room == 5')
room_g=[len(room_g_1),len(room_g_2),len(room_g_3),len(room_g_4),len(room_g_5)]

In [390]:
#numero di volte per ogni valore di room dato bad
room_b_1=data.query('target == " bad" and room == 1')
room_b_2=data.query('target == " bad" and room == 2')
room_b_3=data.query('target == " bad" and room == 3')
room_b_4=data.query('target == " bad" and room == 4')
room_b_5=data.query('target == " bad" and room == 5')
room_b=[len(room_b_1),len(room_b_2),len(room_b_3),len(room_b_4),len(room_b_5)]

In [391]:
#numero di volte per ogni valore di location dato good
loc_g_1=data.query('target == " good" and location == 1')
loc_g_2=data.query('target == " good" and location == 2')
loc_g_3=data.query('target == " good" and location == 3')
loc_g_4=data.query('target == " good" and location == 4')
loc_g_5=data.query('target == " good" and location == 5')
loc_g=[len(loc_g_1),len(loc_g_2),len(loc_g_3),len(loc_g_4),len(loc_g_5)]

In [393]:
#numero di volte per ogni valore di location dato bad
loc_b_1=data.query('target == " bad" and location == 1')
loc_b_2=data.query('target == " bad" and location == 2')
loc_b_3=data.query('target == " bad" and location == 3')
loc_b_4=data.query('target == " bad" and location == 4')
loc_b_5=data.query('target == " bad" and location == 5')
loc_b=[len(loc_b_1),len(loc_b_2),len(loc_b_3),len(loc_b_4),len(loc_b_5)]

In [394]:
#numero di volte per ogni valore di check dato good
check_g_1=data.query('target == " good" and check == 1')
check_g_2=data.query('target == " good" and check == 2')
check_g_3=data.query('target == " good" and check == 3')
check_g_4=data.query('target == " good" and check == 4')
check_g_5=data.query('target == " good" and check == 5')
check_g=[len(check_g_1),len(check_g_2),len(check_g_3),len(check_g_4),len(check_g_5)]

In [395]:
#numero di volte per ogni valore di check dato bad
check_b_1=data.query('target == " bad" and check == 1')
check_b_2=data.query('target == " bad" and check == 2')
check_b_3=data.query('target == " bad" and check == 3')
check_b_4=data.query('target == " bad" and check == 4')
check_b_5=data.query('target == " bad" and check == 5')
check_b=[len(check_b_1),len(check_b_2),len(check_b_3),len(check_b_4),len(check_b_5)]

In [396]:
#numero di volte per ogni valore di service dato good
ser_g_1=data.query('target == " good" and service == 1')
ser_g_2=data.query('target == " good" and service == 2')
ser_g_3=data.query('target == " good" and service == 3')
ser_g_4=data.query('target == " good" and service == 4')
ser_g_5=data.query('target == " good" and service == 5')
ser_g=[len(ser_g_1),len(ser_g_2),len(ser_g_3),len(ser_g_4),len(ser_g_5)]

In [397]:
#numero di volte per ogni valore di service dato bad
ser_b_1=data.query('target == " bad" and service == 1')
ser_b_2=data.query('target == " bad" and service == 2')
ser_b_3=data.query('target == " bad" and service == 3')
ser_b_4=data.query('target == " bad" and service == 4')
ser_b_5=data.query('target == " bad" and service == 5')
ser_b=[len(ser_b_1),len(ser_b_2),len(ser_b_3),len(ser_b_4),len(ser_b_5)]

In [398]:
#numero di volte per ogni valore di business dato good
b_g_1=data.query('target == " good" and business == 1')
b_g_2=data.query('target == " good" and business == 2')
b_g_3=data.query('target == " good" and business == 3')
b_g_4=data.query('target == " good" and business == 4')
b_g_5=data.query('target == " good" and business == 5')
b_g=[len(b_g_1),len(b_g_2),len(b_g_3),len(b_g_4),len(b_g_5)]

In [399]:
#numero di volte per ogni valore di service dato bad
b_b_1=data.query('target == " bad" and business == 1')
b_b_2=data.query('target == " bad" and business == 2')
b_b_3=data.query('target == " bad" and business == 3')
b_b_4=data.query('target == " bad" and business == 4')
b_b_5=data.query('target == " bad" and business == 5')
b_b=[len(b_b_1),len(b_b_2),len(b_b_3),len(b_b_4),len(b_b_5)]

In [380]:
#create view to cpt
def generate_cpt(name,array_g,array_b):
    cpt= PrettyTable()
    cpt.field_names =["P("+name+"|target)",1,2,3,4,5]
    good_c=['good']
    bad_c=['bad']
    for i in range(len(array_g)):
        good_c.append(float(array_g[i])/good_n)
        bad_c.append(float(array_b[i])/bad_n)
    cpt.add_row(good_c)
    cpt.add_row(bad_c)
    return cpt


In [401]:
print(generate_cpt("clean",clean_g,clean_b))

+-----------------+-------------------+------------------+----------------+----------------+-----------------+
| P(clean|target) |         1         |        2         |       3        |       4        |        5        |
+-----------------+-------------------+------------------+----------------+----------------+-----------------+
|       good      | 0.000260395252583 | 0.00320012060412 | 0.031829893375 | 0.290039196338 |  0.67467039443  |
|       bad       |   0.172129124885  |  0.220292976198  | 0.326087444198 | 0.202840029612 | 0.0786504251071 |
+-----------------+-------------------+------------------+----------------+----------------+-----------------+


In [403]:
print(generate_cpt("value",value_g,value_b))

+-----------------+-------------------+------------------+-----------------+-----------------+-----------------+
| P(value|target) |         1         |        2         |        3        |        4        |        5        |
+-----------------+-------------------+------------------+-----------------+-----------------+-----------------+
|       good      | 0.000322067812406 | 0.00359071348299 | 0.0551147109613 |  0.387104952992 |  0.553867554752 |
|       bad       |   0.241424950086  |  0.300895080423  |  0.36384234022  | 0.0787401574803 | 0.0150974717904 |
+-----------------+-------------------+------------------+-----------------+-----------------+-----------------+


In [405]:
print(generate_cpt("room",room_g,room_b))

+----------------+-------------------+-----------------+-----------------+----------------+----------------+
| P(room|target) |         1         |        2        |        3        |       4        |       5        |
+----------------+-------------------+-----------------+-----------------+----------------+----------------+
|      good      | 0.000390592878875 | 0.0074760847518 | 0.0712523641148 | 0.375709234438 | 0.545171723817 |
|      bad       |   0.223927137313  |  0.291248850304 |  0.326087444198 | 0.126500213114 | 0.032236355071 |
+----------------+-------------------+-----------------+-----------------+----------------+----------------+


In [407]:
print(generate_cpt("location",loc_g,loc_b))

+--------------------+-------------------+------------------+----------------+----------------+----------------+
| P(location|target) |         1         |        2         |       3        |       4        |       5        |
+--------------------+-------------------+------------------+----------------+----------------+----------------+
|        good        | 0.000500232985226 | 0.00596853328948 | 0.034372173341 | 0.344872954527 | 0.614286105858 |
|        bad         |   0.157390582587  |  0.227045337282  | 0.333983893039 | 0.167530340759 | 0.114049846333 |
+--------------------+-------------------+------------------+----------------+----------------+----------------+


In [409]:
print(generate_cpt("check",check_g,check_b))

+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+
| P(check|target) |        1         |        2        |        3        |        4        |        5        |
+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+
|       good      | 0.00198037442096 | 0.0090384562673 | 0.0444042430721 |  0.340638105419 |  0.603938820821 |
|       bad       |  0.219104022254  |  0.258698431927 |  0.375911344415 | 0.0997599659017 | 0.0465262355026 |
+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+


In [413]:
print(generate_cpt("service",ser_g,ser_b))

+-------------------+-------------------+------------------+-----------------+----------------+-----------------+
| P(service|target) |         1         |        2         |        3        |       4        |        5        |
+-------------------+-------------------+------------------+-----------------+----------------+-----------------+
|        good       | 0.000925088397336 | 0.00768851245786 | 0.0565811473837 | 0.326542499246 |  0.608262752515 |
|        bad        |   0.234941786123  |  0.253179890975  |  0.331987347735 | 0.136348341073 | 0.0435426340938 |
+-------------------+-------------------+------------------+-----------------+----------------+-----------------+


In [414]:
print(generate_cpt("business",b_g,b_b))

+--------------------+------------------+----------------+-----------------+-----------------+-----------------+
| P(business|target) |        1         |       2        |        3        |        4        |        5        |
+--------------------+------------------+----------------+-----------------+-----------------+-----------------+
|        good        | 0.00322067812406 | 0.012382479511 | 0.0736781514678 |  0.386906230299 |  0.523812460598 |
|        bad         |  0.231105727169  | 0.28537137986  |  0.40579222469  | 0.0558584023151 | 0.0218722659668 |
+--------------------+------------------+----------------+-----------------+-----------------+-----------------+


In [411]:
skel= GraphSkeleton()

In [412]:
skel.load('./skeleton.txt')

In [87]:
skel.toporder()

In [88]:
skel.getparents("room")

[u'target']

In [89]:
skel.getchildren("target")

[u'clean', u'value', u'room', u'location', u'check', u'service', u'business']